In [1]:
import numpy as np
from math import sqrt

In [2]:
# Função para Calcular Flechas

def CalcFlecha(q,V,T,C1):
    fc = C1*(np.cosh((q*V)/(2*T))-1)
    return fc
def CalcComp(e,C1,V):
    e1 = np.zeros(len(e)+1)
    e2 = np.zeros(len(e)+1)
    e1[0] = e[0]
    e2[-1] = e[-1]
    e1[1:] = e1[1:] + e
    e2[:-1] = e2[:-1] + e
    l = np.sqrt((e1-e2)**2 + 4*C1**2*np.sinh(V/(2*C1))**2)
    return l
def CalcH(hmin,fc):
    hmin1 = np.zeros(len(hmin)-1)
    hmin2 = np.zeros(len(hmin)-1)
    hmin1 = hmin[:-1]
    hmin2 = hmin[1:]
    fc1 = np.zeros(len(fc)-1)
    fc2 = np.zeros(len(fc)-1)
    fc1 = fc[:-1]
    fc2 = fc[1:]
    H = np.maximum(hmin1,hmin2)+np.maximum(fc1,fc1)
    return H
def FindTo2(q,E,S,PesoCabo,Ag,TracaoTo,alfa_t,t,t_EDS,Tol):
    A = (E*S*PesoCabo**2*Ag**2)/(24*TracaoTo**2)+E*S*alfa_t*(t-t_EDS)-TracaoTo
    B = (E*S*q**2*Ag**2)/24
    To2 = TracaoTo+TracaoTo*0.1
    mod_erro = 100
    while(mod_erro > Tol):
        erro = -(To2**3+To2**2*A-B)/(3*To2**2+2*To2*A)
        mod_erro = np.max(np.absolute(erro))
        To2 = To2 + erro
    return To2

In [3]:
# Dados dos Cabos
PesoCabo =  1.9801     # kgf/m, Catálogo Nexans
PesoCabo

1.9801

In [4]:
TensaoRuptura = 16125       # retirado do Catálogo Nexans
TensaoRuptura

16125

In [5]:
TracaoTo = 0.2*TensaoRuptura    # Máxima tensão de ruptura adm. (NBR5422)
TracaoTo

3225.0

In [6]:
C1 = TracaoTo/PesoCabo  # Constante de Contorno
C1

1628.705620928236

In [7]:
E = 67e+3   # Módulo de elasticidade para cabos CAA 54/7 # catálogo Alubrás
E

67000.0

In [8]:
alfa_t = 19.4e-6 # Coef. dilatação para cabos CAA 54/7 # catálogo Alubrás
alfa_t

1.94e-05

In [9]:
S = 591.26 # Área de Secção Transversal em mm²  Catálogo Nexans
S

591.26

In [10]:
diametro = 31.62/1000 # Diâmetro do condutor em m,  Catálogo Nexans
diametro

0.03162

In [11]:
# Dados do Projeto

x = np.zeros(8)
x[0] = 150        # Distância entre o centro do Vão 1 e Torre E1
x[1] = 380        # Distância entre a Torre E1 e o centro do Vão 2
x[2] = 100        # Distância entre a Torre E2 e o centro do Vão 2
x[3] = 250        # Distância entre o centro do Vão 2 e o centro do Vão 3
x[4] = 50         # Distância entre o centro do Vão 3 e Torre E3
x[5] = 200        # Distância entre a Torre E3 e o centro do Vão 4
x[6] = 150        # Distância entre o centro do Vão 4 e Torre E4
x[7] = 250        # Distância entre a Torre E4 e o centro do Vão 5

V = np.zeros(5)
V[0] = 300        # Centro do vão 1
V[1] = 280        # Centro do vão 2
V[2] = 400        # Centro do vão 3
V[3] = 350        # Centro do vão 4
V[4] = 500        # Centro do vão 5

E =  np.zeros(4)
E[0] = 50         # Altura de desnível da Torre E1
E[1] = 20         # Altura de desnível da Torre E2
E[2] = 10         # Altura de desnível da Torre E3
E[3] = 0          # Torre de referência, considerando 0 m de desnível

Vlinha = 500    # Tensão da LT, em kV
llinha = 430e+3 # Comprimento da LT, em kV]

In [12]:
# Dados do Ambiente %

alt = 430           # Altitude média do ambiente
t_EDS = 23          # Temperatura EDS
t_min = 1.2         # Temperatura mínima
vt_min = 25         # Velocidade do vento p/ temperatura mínima
vt_min_ang = 0      # Ângulo de incidência do vento p/ temp. mínima
t_coin = 18.4       # Temperatura coincidente
vt_coin = 32        # Velocidade do vento p/ temperatura coincidente
vt_coin_ang = 90    # Ângulo de incidência do vento p/ temp. coincidente
t_max = 44          # Temperatura máxima
vt_max = 0          # Velocidade do vento p/ temperatura máxima
vt_max_ang = 0      # Ângulo de incidência do vento p/ temp. máxima

In [13]:
# Variáveis de Uso Comum

Tol = 1e-6                                                                      # Tolerância para divergência
print('Tol:',Tol)
par = (1.293/(1+0.00367*t_coin))*((16000+64*t_coin-alt)/(16000+64*t_coin+alt))  # Massa específica do ar, em kg/m3
print('par:',par)
Cxc = 1.0                                                                       # Coeficiente de arrasto (padrão: 1.0)
alfa_e = 0.8                                                                    # coeficiente de efetividade da pressão do vento (padrão: 0.8)
size = 5
q = np.zeros(size)
fv = np.zeros(size)
q[0] = PesoCabo
fv[0] = 0

Tol: 1e-06
par: 1.1520508960635631


In [14]:
# Alturas Míninas dos Vãos

hmin_v = np.zeros(5)
hmin_v[0] = 8.0+0.01*(Vlinha/sqrt(3)-50)    # Altura p/ Rodovia Federal
hmin_v[1] = 8.0+0.01*(Vlinha/sqrt(3)-50)    # Altura p/ Rodovia Federal
hmin_v[2] = 9.0+0.01*(Vlinha/sqrt(3)-50)    # Altura p/ Ferrovia ñ Eletrificada
hmin_v[3] = 6.5+0.01*(Vlinha/sqrt(3)-50)    # Altura p/ Máquinas Agrícolas
hmin_v[4] = 6.5+0.01*(Vlinha/sqrt(3)-50)    # Altura p/ Máquinas Agrícolas
hmin_v

array([10.38675135, 10.38675135, 11.38675135,  8.88675135,  8.88675135])

In [15]:
# Cálculo do Vão Médio das Torres
# Vão médio (am ou vm): representa a média aritmética entre os vãos adjacentes à estrutura.

am_E = np.zeros(4)
am_E[0] = (2*x[0]+(x[1]-x[2]))/2        # Vão 1 está entre torre E-∞ e E1
am_E[1] = ((x[1]+x[3]+x[4]))/2          # Vão 2 está entre torre E1 e E2
am_E[2] = (x[2]+x[3]+x[4]+x[5]+x[6])/2  # Vão 3 está entre torre E3 e E4
am_E[3] = ((x[5]+x[6])+2*x[7])/2        # Vão 4 está entre torre E4 e E+∞
am_E

array([290., 340., 375., 425.])

In [16]:
# Cálculo do Vão Gravante das Torres
# Vão gravante (Ag ou Vg): somatório das distâncias da estrutura aos centros das catenárias adjacentes a ela.

Ag_E = np.zeros(4)
Ag_E[0] = x[0]+x[1] # Distância entre centros do vãos 1 e 2
Ag_E[1] = x[3]      # Distância entre centros do vãos 2 e 3
Ag_E[2] = x[4]+x[5] # Distância entre centros do vãos 3 e 4
Ag_E[3] = x[6]+x[7] # Distância entre centros do vãos 4 e 5
Ag_E

array([530., 250., 250., 400.])

In [17]:
# Cálculo do Vão Regulador

# regulador (ar ou vr): representa a forma das catenárias de todos os vãos do trecho projetado.

a = np.zeros(5)
a[0] = 2*x[0]                                               # Distância entre Torre E-∞ e Torre E1
a[1] = x[1] - x[2]                                          # Distância entre Torre E1 e Torre E2
a[2] = x[2] + x[3] + x[4]                                   # Distância entre Torre E2 e Torre E3
a[3] = x[5] + x[6]                                          # Distância entre Torre E3 e Torre E4
a[4] = 2*x[7]                                               # Distância entre Torre E4 e Torre E+∞
ar = sqrt(np.sum(a**3)/np.sum(a))  #Equação de ar
ar

391.7363616632752

In [18]:
'''
Trações nos Cabos
As trações verticais em cada estrutura são sempre dadas pelo peso do condutor (sempre em [kgf/m]) multiplicadas pelo somatório das distâncias entre a estrutura
e os vértices das catenárias adjacentes a ela, ou seja, peso multiplicado pelo vão gravante (Vg).
'''
Tv_E = np.zeros(4)
T_E = np.zeros(4)

Tv_E[0] = PesoCabo*Ag_E[0]              # Tração vertical da Torre E1
T_E[0] = sqrt(Tv_E[0]**2+TracaoTo**2)   # Tração total da Torre E1
Tv_E[1] = PesoCabo*Ag_E[1]              # Tração vertical da Torre E2
T_E[1] = Tv_E[1]                        # Tração total da Torre E2
Tv_E[2] = PesoCabo*Ag_E[2]              # Tração vertical da Torre E3
T_E[2] = Tv_E[2]                        # Tração total da Torre E3
Tv_E[3] = PesoCabo*Ag_E[3]              # Tração vertical da Torre E4
T_E[3] = sqrt(Tv_E[3]**2+TracaoTo**2)   # Tração total da Torre E4
print(Tv_E)
print(T_E)

[1049.453  495.025  495.025  792.04 ]
[3391.45641269  495.025       495.025      3320.83609376]


In [19]:
'''
Cálculo das Flechas para Condição Inicial
A equação da catenária é a equação geral para o comprimento dos cabos e pode ser usada tanto para cabos suspensos por estruturas de mesma altura quando para cabos
suspensos por estruturas de alturas diferentes.
'''
fc = np.zeros([4,5])
fc[0] = CalcFlecha(PesoCabo,V,TracaoTo,C1) # Flecha de todos os vãos
fc

array([[ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [20]:
'''
Cálculo do Comprimento da Linha para Condição Inicial
A equação da catenária é a equação geral para o comprimento dos cabos e pode ser
usada tanto para cabos suspensos por estruturas de mesma altura quando para cabos
suspensos por estruturas de alturas diferentes.
'''
l = CalcComp(E,C1,V) # Comprimento da linha entre E-∞ e E1, nivelada
l_tot = np.zeros(size)
l_tot[1] = np.sum(l)
l_tot

array([   0.        , 1836.28264013,    0.        ,    0.        ,
          0.        ])

In [21]:
'''
Cálculo de Altura da Torre para Condição Inicial
A altura da torre é dada pela soma entre a altura mínima e a flecha.
'''
H_E = CalcH(hmin_v,fc[0])
print(H_E)
Toc = np.zeros(size)
Toc[0] = TracaoTo

[17.29896063 17.40750519 23.68187956 18.29743752]


In [22]:
# Correção para Temperatura Máxima (44oC) Sem Vento

q0 = 1/2*par*vt_max**2
#fv = q0*Cxc*alfa_e*diametro*0.102*sind(vt_coin_ang)^2;
fv[1] = 0.0045*vt_max**2*alfa_e*diametro
q[1] = np.sqrt(fv[1]**2+PesoCabo**2)

In [23]:
# Novas Trações dos Cabos

To2 = FindTo2(q[1],E,S,PesoCabo,Ag_E,TracaoTo,alfa_t,t_max,t_EDS,Tol)
print(To2, len(To2))

[3213.86171791 3220.21692523 3222.59985613 3225.        ] 4


In [24]:
# Novas Flechas
C1 = np.max(To2)/q[1]
fc[1] = CalcFlecha(q[1],V,np.max(To2),C1) # Flecha de todos os vãos
fc

array([[ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [25]:
# Novo Comprimento das Linhas

l = CalcComp(E,C1,V) # Comprimento da linha entre E-∞ e E1, nivelada
l_tot[1] = np.sum(l)
l_tot[1]

1836.2826401309699

In [26]:
# Novas Alturas das Torres

H_E = CalcH(hmin_v,fc[1])
Toc[1] = np.max(To2)
print(H_E,Toc)

[17.29896063 17.40750519 23.68187956 18.29743752] [3225. 3225.    0.    0.    0.]


In [27]:
# Correção para Temperatura Mínima (1,2 oC) Com Vento de 25 km/h

q0 = 1/2*par*vt_min**2
fv[2] = 0.0045*vt_min**2*alfa_e*diametro;
q[2] = sqrt(fv[2]**2 + PesoCabo**2);

In [28]:
# Novas Trações dos Cabos

To2 = FindTo2(q[2],E,S,PesoCabo,Ag_E,TracaoTo,alfa_t,t_min,t_EDS,Tol)
print(To2, len(To2))

[3236.72652663 3229.98028229 3227.49905324 3225.        ] 4


In [29]:
# Novas Flechas
C1 = np.max(To2)/q[2]
fc[2] = CalcFlecha(q[2],V,np.max(To2),C1) # Flecha de todos os vãos
fc

array([[ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 6.89158176,  6.00278987, 12.25839681,  9.38258841, 19.16720335],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [30]:
# Novo Comprimento da Linha

l = CalcComp(E,C1,V) # Comprimento da linha entre E-∞ e E1, nivelada
l_tot = np.sum(l)
l_tot

1836.256359640742

In [31]:
# Novas Alturas das Torres
H_E = CalcH(hmin_v,fc[2])
Toc[2] = np.max(To2)
print(H_E,Toc)

[17.2783331  17.38954122 23.64514816 18.26933976] [3225.         3225.         3236.72652663    0.            0.        ]


In [32]:
# Correção para Temperatura Coincidente (18,4 oC) Com Vento de 32 km/h e 90°

q0 = 1/2*par*vt_coin**2;
#fv = q0*Cxc*alfa_e*diametro*0.102*sind(vt_coin_ang)^2;
fv[3] = 0.0045*vt_coin**2*alfa_e*diametro
q[3] = sqrt(fv[3]**2 + PesoCabo**2)

In [33]:
# Novas Trações dos Cabos

To2 = FindTo2(q[3],E,S,PesoCabo,Ag_E,TracaoTo,alfa_t,t_coin,t_EDS,Tol)
print(To2, len(To2))

[3227.8585005  3226.08765695 3225.54578431 3225.        ] 4


In [34]:
# Novas Flechas
C1 = np.max(To2)/q[3]
fc[3] = CalcFlecha(q[3],V,np.max(To2),C1) # Flecha de todos os vãos
fc

array([[ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 6.91220928,  6.02075385, 12.29512821,  9.41068617, 19.22471731],
       [ 6.89158176,  6.00278987, 12.25839681,  9.38258841, 19.16720335],
       [ 6.91805214,  6.02584223, 12.30553265,  9.41864504, 19.24100868]])

In [35]:
# Novo Comprimento da Linha

l = CalcComp(E,C1,V) # Comprimento da linha entre E-∞ e E1, nivelada
l_tot = np.sum(l)
l_tot

1836.2900984658731

In [36]:
# Novas Alturas das Torres

H_E = CalcH(hmin_v,fc[2])
Toc[3] = np.max(To2)


In [37]:
print(q)
print(fv)
print(fc)

[1.9801     1.9801     1.98137771 1.98352796 0.        ]
[0.         0.         0.071145   0.11656397 0.        ]
[[ 6.91220928  6.02075385 12.29512821  9.41068617 19.22471731]
 [ 6.91220928  6.02075385 12.29512821  9.41068617 19.22471731]
 [ 6.89158176  6.00278987 12.25839681  9.38258841 19.16720335]
 [ 6.91805214  6.02584223 12.30553265  9.41864504 19.24100868]]


In [38]:
H_E

array([17.2783331 , 17.38954122, 23.64514816, 18.26933976])